Welcome to the Wallaroo `ccfraud` model example!  This example will demonstrate how to use Wallaroo to detect credit card fraud through a trained model and sample data.  By the end of this example, you'll be able to:

* Start the Wallaroo client.
* Create a workspace.
* Upload our model to the workspace.
* Create a new pipeline and set it to our credit card fraud model.
* Run a smoke test to verify the pipeline and model is working properly.
* Perform an inference and display the results

The first thing we'll do is launch an instance of the Wallaroo Client and authenticate to Wallaroo.

In [1]:
import wallaroo
import os


In [2]:
os.environ["WALLAROO_SDK_CREDENTIALS"] = 'creds.json'
wl = wallaroo.Client(auth_type="user_password")

Next we're going to create a new workspace for our model, then set it as our current workspace context.  We'll list all of the current workspaces, create our new workspace, then display a list of workspaces again so you can see it's been added.

In [3]:
wl.list_workspaces()


Name,Created At,Users,Models,Pipelines
New Project Workspace,2022-03-24 19:49:51,['steve@ex.co'],1,1
ccfraud,2022-03-24 19:59:28,['steve@ex.co'],1,1
ccfraud-workspace,2022-03-25 15:24:10,['steve@ex.co'],0,0
ccfraud-workspace,2022-03-25 15:45:32,['steve@ex.co'],1,1
ccfraud-workspace,2022-03-25 16:26:50,['steve@ex.co'],1,1


In [4]:
new_workspace = wl.create_workspace("ccfraud-workspace")
_ = wl.set_current_workspace(new_workspace)

Just to make sure, let's list our current workspace.  If everything is going right, it will show us we're in the `ccfraud-workspace`.

In [5]:
wl.list_workspaces()

Name,Created At,Users,Models,Pipelines
New Project Workspace,2022-03-24 19:49:51,['steve@ex.co'],1,1
ccfraud,2022-03-24 19:59:28,['steve@ex.co'],1,1
ccfraud-workspace,2022-03-25 15:24:10,['steve@ex.co'],0,0
ccfraud-workspace,2022-03-25 15:45:32,['steve@ex.co'],1,1
ccfraud-workspace,2022-03-25 16:26:50,['steve@ex.co'],1,1
ccfraud-workspace,2022-03-25 16:37:50,['steve@ex.co'],0,0


Our workspace is created.  Let's upload our credit card fraud model to it.  This is the file name `ccfraud.onnx`, and we'll upload it as `ccfraud-model`.  Since we're already in our default workspace `ccfraud-workspace`, it'll be uploaded right to there.  Once that's done uploading, we'll list out all of the models currently deployed so we can see it included.

In [6]:
# Now we should be able to upload this model.
model = wl.upload_model("ccfraud-model", "./ccfraud.onnx").configure()

In [7]:
# Display the models
for m in wl.list_models():
    print("model name: " + m.name())
    print(str(m.last_update_time()))
    print("----------------------")

model name: ccfraud-model
2022-03-25 16:37:51.331417+00:00
----------------------
model name: ccfraud-model
2022-03-25 16:27:27.443729+00:00
----------------------
model name: ccfraud-model
2022-03-25 15:45:43.445335+00:00
----------------------
model name: kerasccfraud
2022-03-24 20:03:13.703624+00:00
----------------------
model name: aloha-2
2022-03-24 19:51:00.928787+00:00
----------------------


With our model uploaded, time to create our pipeline to deploy the model so it can accept data and act on it.  We'll call our pipeline `ccfraud-pipeline`.  This takes about 45 seconds, and when finished it will display `ok` at the end.

In [8]:
# Create the pipeline and deploy it
p = wl.build_pipeline('ccfraud-pipeline')
p = p.add_model_step(model)
pipeline = p.deploy()

Waiting for deployment - this will take up to 45s ....... ok


we can see our new pipeline with the `pipelines_by_name` command.  Note that if you already have a pipeline named `ccfraud-pipeline` you'll have to update the array number.

In [9]:
wl.pipelines_by_name('ccfraud-pipeline')[1].status()

{'status': 'Running',
 'details': None,
 'engines': [{'ip': '10.12.1.179',
   'name': 'engine-6c45c5cb6b-chq8c',
   'status': 'Running',
   'reason': None,
   'pipeline_statuses': {'pipelines': [{'id': 'ccfraud-pipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'ccfraud-model',
      'version': '31d8c19a-e01f-4f04-abc6-17a3543a95a7',
      'sha': 'bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.12.1.180',
   'name': 'engine-lb-85846c64f8-7clrb',
   'status': 'Running',
   'reason': None}]}

With our pipeline deployed, let's run a smoke test to make sure it's working right.  We'll run an inference through our pipeline from the file `smoke_test.json` and see the results.

In [10]:
# Inference from file the way the demo does it
pipeline.infer_from_file('./smoke_test.json')

Waiting for inference response - this will take up to 45s ... ok


[InferenceResult({'check_failures': [],
  'elapsed': 185745,
  'model_name': 'ccfraud-model',
  'model_version': 'f4024eed-d75c-402e-9d4a-f48b48ec0070',
  'original_data': {'tensor': [[1.0678324729342086,
                                0.21778102664937624,
                                -1.7115145261843976,
                                0.6822857209662413,
                                1.0138553066742804,
                                -0.43350000129006655,
                                0.7395859436561657,
                                -0.28828395953577357,
                                -0.44726268795990787,
                                0.5146124987725894,
                                0.3791316964287545,
                                0.5190619748123175,
                                -0.4904593221655364,
                                1.1656456468728569,
                                -0.9776307444180006,
                                -0.6322198962519854,
    

Looks good!  Time to run the real test on some real data.  Run another inference this time from the file `high_fraud.json` and let's ee the results:

In [11]:
pipeline.infer_from_file('./high_fraud.json')

[InferenceResult({'check_failures': [],
  'elapsed': 161122,
  'model_name': 'ccfraud-model',
  'model_version': 'f4024eed-d75c-402e-9d4a-f48b48ec0070',
  'original_data': {'tensor': [[1.0678324729342086,
                                18.155556397512136,
                                -1.658955105843852,
                                5.2111788045436445,
                                2.345247064454334,
                                10.467083577773014,
                                5.0925820522419745,
                                12.82951536371218,
                                4.953677046849403,
                                2.3934736228338225,
                                23.912131817957253,
                                1.7599568310350209,
                                0.8561037518143335,
                                1.1656456468728569,
                                0.5395988813934498,
                                0.7784221343010385,
                  

And there we have it!  Feel free to use this as a template for other models, inferences and pipelines that you want to deploy with Wallaroo!